In [1]:
! pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


#Obtiene todos los conceptos del vocabulario de la UNESCO y sus clasificaciones superiores en `idioma español`

ejemplo: el concepto `Profesión` con uri http://vocabularies.unesco.org/thesaurus/concept678 es clasificado o agrupado por `Política, derecho y economía` y `Gestión de personal`

In [3]:
from rdflib import Graph, URIRef
import json
from pprint import pprint

g = Graph()
g.parse("/content/unesco-thesaurus.ttl")

knows_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX isothes: <http://purl.org/iso25964/skos-thes#>
PREFIX unesco: <http://vocabularies.unesco.org/ontology#>
SELECT DISTINCT ?concept ?conceptLabel ?group ?groupLabel ?domain ?domainLabel
WHERE {
  # obtiene los dominios de la ontologia
  ?domain a unesco:Domain .
  # de esos dominios obtener sus miembros que en este caso son las agrupaciones intermedias
  ?domain skos:member ?group .
  # esos miembros son clasificados como ConceptGroup
  ?group a isothes:ConceptGroup  .
  # de esos grupos obtener sus miembros que son los conceptos
  ?group skos:member ?concept .
  # comprobar que son conceptos
  ?concept a skos:Concept .

  # obtener la propiedad Label de los dominios y filtrar por el español
  ?domain skos:prefLabel ?domainLabel
  FILTER(LANGMATCHES(LANG(?domainLabel), "es"))
  # obtener la propiedad Label de los grupos y filtrar por el español
  ?group skos:prefLabel ?groupLabel
  FILTER(LANGMATCHES(LANG(?groupLabel), "es"))
  # obtener la propiedad Label de los conceptos y filtrar por el español
  ?concept skos:prefLabel ?conceptLabel
  FILTER(LANGMATCHES(LANG(?conceptLabel), "es"))

  }


"""
# esta consulta sparql retorna una lista, cada elemento contiene una lista con 6 elementos
# posiciones |  significado
# 0          |  URI del concepto
# 1          |  concept Label
# 2          |  URI del grupo que calsifica al concepto
# 3          |  group Label
# 4          |  URI del dominio que calsifica al grupo
# 5          |  domain Label
qres = g.query(knows_query)
count = 0
query_format = []

for row in qres:
    query_format.append(
        {'str': "El concepto " + row[1] +" está relacionado a " + row[3]+", "+row[5],
         'conceptUri': row[0].toPython(),
         'conceptLabel': row[1],
         'belongsTo': [
             {
                 'group':{
                    'uri': row[2].toPython(),
                    'label': row[3]
                 },
                 'domain': {
                     'uri': row[4].toPython(),
                     'label': row[5]
                 }
             }
          ]
        }
      )
    count += 1
    # if count == 20: break

pprint(count)

with open('/content/unesco-parser.json', 'w', encoding='utf-8') as f:
    json.dump(query_format, f, ensure_ascii=False, indent=4)

4479


#Obtiene todos los conceptos del vocabulario de la UNESCO y sus clasificaciones superiores en `idioma ingles`


In [2]:
from rdflib import Graph, URIRef
import json
from pprint import pprint

g = Graph()
g.parse("/content/unesco-thesaurus.ttl")

knows_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX isothes: <http://purl.org/iso25964/skos-thes#>
PREFIX unesco: <http://vocabularies.unesco.org/ontology#>
SELECT DISTINCT ?concept ?conceptLabel ?group ?groupLabel ?domain ?domainLabel
WHERE {
  # obtiene los dominios de la ontologia
  ?domain a unesco:Domain .
  # de esos dominios obtener sus miembros que en este caso son las agrupaciones intermedias
  ?domain skos:member ?group .
  # esos miembros son clasificados como ConceptGroup
  ?group a isothes:ConceptGroup  .
  # de esos grupos obtener sus miembros que son los conceptos
  ?group skos:member ?concept .
  # comprobar que son conceptos
  ?concept a skos:Concept .

  # obtener la propiedad Label de los dominios y filtrar por el español
  ?domain skos:prefLabel ?domainLabel
  FILTER(LANGMATCHES(LANG(?domainLabel), "en"))
  # obtener la propiedad Label de los grupos y filtrar por el español
  ?group skos:prefLabel ?groupLabel
  FILTER(LANGMATCHES(LANG(?groupLabel), "en"))
  # obtener la propiedad Label de los conceptos y filtrar por el español
  ?concept skos:prefLabel ?conceptLabel
  FILTER(LANGMATCHES(LANG(?conceptLabel), "en"))

  }


"""
# esta consulta sparql retorna una lista, cada elemento contiene una lista con 6 elementos
# posiciones |  significado
# 0          |  URI del concepto
# 1          |  concept Label
# 2          |  URI del grupo que calsifica al concepto
# 3          |  group Label
# 4          |  URI del dominio que calsifica al grupo
# 5          |  domain Label
qres = g.query(knows_query)
count = 0
query_format = []

for row in qres:
    query_format.append(
        {'str': "the concept " + row[1] +" is related to " + row[3]+", "+row[5],
         'conceptUri': row[0].toPython(),
         'conceptLabel': row[1],
         'belongsTo': [
             {
                 'group':{
                    'uri': row[2].toPython(),
                    'label': row[3]
                 },
                 'domain': {
                     'uri': row[4].toPython(),
                     'label': row[5]
                 }
             }
          ]
        }
      )
    count += 1
    # if count == 20: break

pprint(count)

with open('/content/unesco-parser-en.json', 'w', encoding='utf-8') as f:
    json.dump(query_format, f, ensure_ascii=False, indent=4)

4479
